# Load Data

In [41]:
import pandas as pd

data = pd.read_csv('data_processed.csv', encoding="latin-1")
data.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,...,sambareggae,sangueo,secondline,shuffle,slow,soft,songo,soul,swing,venezuelan
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,1,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,...,0,0,0,0,0,0,0,0,0,0
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,1,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,...,0,0,0,0,0,0,0,1,0,0
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,1,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,...,0,0,0,0,0,0,0,0,0,0
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,1,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,...,0,0,0,0,0,0,0,1,0,0
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,1,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,...,0,0,0,0,0,0,0,1,0,0


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 100 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   drummer                  1090 non-null   object 
 1   session                  1090 non-null   object 
 2   id                       1090 non-null   object 
 3   style                    1090 non-null   object 
 4   bpm                      1090 non-null   int64  
 5   beat_type                1090 non-null   int64  
 6   time_signature           1090 non-null   object 
 7   midi_filename            1090 non-null   object 
 8   audio_filename           1090 non-null   object 
 9   duration                 1090 non-null   float64
 10  split                    1090 non-null   object 
 11  onset_env_mean           1090 non-null   float64
 12  onset_env_std            1090 non-null   float64
 13  mfcc_mean                1090 non-null   float64
 14  mfcc_std               

# Split labels and features

In [43]:
# add the column spectrum_filename to the dataframe data. But not at the end, instead it should follow the column audio_filename
data.insert(data.columns.get_loc('audio_filename') + 1, 'spectrum_filename', '')

In [44]:
data['spectrum_filename'] = data.audio_filename.str.replace('.wav', '.png')

In [45]:
data['spectrum_filename'][0]

'drummer1/eval_session/1_funk-groove1_138_beat_4-4.png'

In [46]:
data.iloc[:, 31:]

,tempogram_std,afrobeat,afrocuban,ando,baiao,bembe,blues,bomba,bossa,brazilian,...,sambareggae,sangueo,secondline,shuffle,slow,soft,songo,soul,swing,venezuelan
0,0.137067,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.163539,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.130047,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.174764,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0.149107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,0.158753,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1086,0.136667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1087,0.176834,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1088,0.160704,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
label_columns = data.columns[32:]
label_columns

Index(['afrobeat', 'afrocuban', 'ando', 'baiao', 'bembe', 'blues', 'bomba',
       'bossa', 'brazilian', 'breakbeat', 'calypso', 'chacarera', 'chacha',
       'country', 'dance', 'disco', 'dominican', 'fast', 'folk', 'frevo',
       'funk', 'fusion', 'gospel', 'groove1', 'groove10', 'groove2', 'groove3',
       'groove4', 'groove5', 'groove6', 'groove7', 'groove8', 'groove9',
       'halftime', 'hiphop', 'ijexa', 'indie', 'jazz', 'joropo', 'klezmer',
       'latin', 'linear', 'maracatu', 'march', 'mediumfast', 'merengue',
       'middleeastern', 'motown', 'neworleans', 'pop', 'prog', 'punk',
       'purdieshuffle', 'reggae', 'reggaeton', 'rhumba', 'rock', 'rockabilly',
       'samba', 'sambareggae', 'sangueo', 'secondline', 'shuffle', 'slow',
       'soft', 'songo', 'soul', 'swing', 'venezuelan'],
      dtype='object')

In [48]:
from sklearn.model_selection import train_test_split

# Split the data
data_train, data_test, = train_test_split(data, test_size=0.2, random_state=42)

# Convolutional Neural Network

In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

path_images = os.path.join('Datasets', 'spectrums')

# Normalize images
image_generator = ImageDataGenerator(
    rescale=1.0/255
)

# Define the data generators
train_generator = image_generator.flow_from_dataframe(
    dataframe=data_train,
    directory=path_images,
    x_col="spectrum_filename",
    y_col=label_columns,
    target_size=(320, 320),
    batch_size=32,
    class_mode="raw",
    color_mode="rgb" # add color mode
)

val_generator = image_generator.flow_from_dataframe(
    dataframe=data_test,
    directory=path_images,
    x_col="spectrum_filename",
    y_col=label_columns,
    target_size=(320, 320),
    batch_size=32,
    class_mode="raw",
    color_mode="rgb", #add color mode,
    shuffle=False,  # this is crucial for later evaluation!
)

Found 178 validated image filenames.
Found 47 validated image filenames.


c:\Users\kevin\miniconda3\envs\audio_data_science_gpu\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 694 invalid image filename(s) in x_col="spectrum_filename". These filename(s) will be ignored.
  warnings.warn(
c:\Users\kevin\miniconda3\envs\audio_data_science_gpu\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 171 invalid image filename(s) in x_col="spectrum_filename". These filename(s) will be ignored.
  warnings.warn(
